# 时区

## 关于时区

### GMT

GMT (Greenwich Mean Time)，格林威治平均时间。

基准：太阳每天经过位于英国伦敦郊区的皇家格林威治天文台的时间为当日中午12点。

### UTC

UTC (Coodinated Universal Time)，协调世界时，又称世界统一时间、世界标准时间、国际协调时间。由于英文（CUT）和法文（TUC）的缩写不同，作为妥协，简称UTC。

UTC 是现在全球通用的时间标准，全球各地都同意将各自的时间进行同步协调。UTC 时间是经过平均太阳时（以格林威治时间GMT为准）、地轴运动修正后的新时标以及以秒为单位的国际原子时所综合精算而成。

### GMT vs UTC

GMT是前世界标准时，UTC是现世界标准时。UTC 比 GMT更精准。

在不需要精确到秒的情况下，二者可以视为等同。

### LMT

LMT (Local Mean Time)，本地平均时。Linux 系统中，选择时区的时候往往只有 Asia/Shanghai 可选而不是 Beijing。上海本地时间比北京本地时间快了 6 分钟。

### CST

CST (Central Standard Time)，中央标准时。


## 坑

但凡是人类妥协而成的，基本在技术上就是个坑。比如中日韩三国文字的Unicode问题。

时间处理是另一个大坑。

这个坑有多大，取决于要处理到什么精度，要解决什么问题。

如果涉及外盘，会有美国夏令时问题。

好在我们只是炒炒期货，不是放火箭，坑还是可以爬的出来的。


## 实践

对于 pandas，文档有 pandas.DatetimeIndex.tz_localize 和 pandas.DatetimeIndex.tz_convert 两条指令。

前者用于将没有指定时区（naive）的时间附加上时区（aware），后者用于转化时区。

pandas 对时区的要求是：str、pytz.timezone 或者 dateutil.tz.tzfile。

pytz 也是 pandas 的需求之一，装了 pandas 必然有 pytz。

以下尝试使用 pytz 库。

In [1]:
# 看看 pytz 都有哪些时区。

import pytz

for tz in pytz.all_timezones:
    print(tz)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

In [2]:
from pathlib import Path

import pandas as pd

from src.utility import PACKAGE_PATH


# 定义数据文件
data_file: str = 'SHFE.al2111_Tick.csv'

# 转为Path格式。
data_path: Path = PACKAGE_PATH.joinpath('data', data_file)

# 加载tick数据为DataFrame。
# 【parse_dates=['datetime']】表示在加载时将 datetime 字段解读为 datetime64 格式。
# 【index_col=['datetime']】表示在加载时将 datetime 字段作为 DateTimeIndex。
df_origin: pd.DataFrame = pd.read_csv(data_path, parse_dates=['datetime'], index_col=['datetime'])

# 显示 df。
print(df_origin.head(20))

                               datetime_nano  last_price  highest   lowest  \
datetime                                                                     
2021-06-30 18:51:42.300  1625050302300000000         NaN      NaN      NaN   
2021-06-30 20:59:00.500  1625057940500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:00.500  1625058000500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.000  1625058001000000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.500  1625058001500000000     18780.0  18785.0  18780.0   
2021-06-30 21:00:02.000  1625058002000000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:02.500  1625058002500000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:03.000  1625058003000000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:03.500  1625058003500000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:04.000  1625058004000000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:04.500  1625058004500000000     18780.0  18795.

In [3]:
import datetime as dt


# 定义时区。
tz_beijing: pytz.BaseTzInfo = pytz.timezone('Etc/GMT-8')
tz_trading: pytz.BaseTzInfo = pytz.timezone('Etc/GMT-12')       # 为什么是GMT-12？想不明白。

# 时区的 delta。
tz_delta: dt.timedelta = dt.timedelta(hours=4)

df = df_origin.copy()

# 将 navie 类型（无时区）的 DateTimeIndex 转化为 aware 类型（时区感知）。
df.index = df.index.tz_localize(tz_beijing)
print(df)

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-06-30 18:51:42.300000+08:00  1625050302300000000         NaN      NaN   
2021-06-30 20:59:00.500000+08:00  1625057940500000000     18780.0  18780.0   
2021-06-30 21:00:00.500000+08:00  1625058000500000000     18780.0  18780.0   
2021-06-30 21:00:01+08:00         1625058001000000000     18780.0  18780.0   
2021-06-30 21:00:01.500000+08:00  1625058001500000000     18780.0  18785.0   
...                                               ...         ...      ...   
2021-09-25 00:59:58+08:00         1632502798000000000     22945.0  23150.0   
2021-09-25 00:59:58.500000+08:00  1632502798500000000     22945.0  23150.0   
2021-09-25 00:59:59+08:00         1632502799000000000     22945.0  23150.0   
2021-09-25 00:59:59.500000+08:00  1632502799500000000     22945.0  23150.0   
2021-09-25 00:59:59.500001+08:00  1632502799500001000     22945.

In [4]:
# 转换为交易时区
df.index = df.index.tz_convert(tz_trading)
print(df)

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-06-30 22:51:42.300000+12:00  1625050302300000000         NaN      NaN   
2021-07-01 00:59:00.500000+12:00  1625057940500000000     18780.0  18780.0   
2021-07-01 01:00:00.500000+12:00  1625058000500000000     18780.0  18780.0   
2021-07-01 01:00:01+12:00         1625058001000000000     18780.0  18780.0   
2021-07-01 01:00:01.500000+12:00  1625058001500000000     18780.0  18785.0   
...                                               ...         ...      ...   
2021-09-25 04:59:58+12:00         1632502798000000000     22945.0  23150.0   
2021-09-25 04:59:58.500000+12:00  1632502798500000000     22945.0  23150.0   
2021-09-25 04:59:59+12:00         1632502799000000000     22945.0  23150.0   
2021-09-25 04:59:59.500000+12:00  1632502799500000000     22945.0  23150.0   
2021-09-25 04:59:59.500001+12:00  1632502799500001000     22945.

In [5]:
print(df.index[1])
print(df.index[1].time())
print(df.index[1].time().tzinfo)

2021-07-01 00:59:00.500000+12:00
00:59:00.500000
None


零散的时间的转换：

In [6]:
t1: dt.time = dt.time(hour=21, minute=0, tzinfo=dt.timezone(dt.timedelta(hours=8)))
t2: dt.time = dt.time(hour=21, minute=0, tzinfo=tz_beijing)
t3: dt.time = pytz.timezone('Asia/Shanghai').localize(
    dt.datetime.combine(
        dt.date.today(),
        dt.time(hour=21, minute=0)
    )
).time()

print(f't1={t1}, t2={t2}, t3={t3}')
print(f'timezone of: t1={t1.tzinfo}, t2={t2.tzinfo}, t3={t3.tzinfo}')
print(type(t1), type(t2), type(t3))

t1=21:00:00+08:00, t2=21:00:00+08:00, t3=21:00:00
timezone of: t1=UTC+08:00, t2=Etc/GMT-8, t3=None
<class 'datetime.time'> <class 'datetime.time'> <class 'datetime.time'>


In [7]:
def convert_time(t: dt.time, tz: dt.timezone) -> dt.time:
    temp: dt.datetime = dt.datetime.combine(
        dt.date.today(),
        t
    ).astimezone(tz=pytz.timezone('Etc/GMT-8'))
    return temp.astimezone(tz).time()
t1: dt.time = dt.time(hour=21, minute=0, tzinfo=dt.timezone(dt.timedelta(hours=8)))
t2: dt.time = dt.time(hour=21, minute=0, tzinfo=pytz.timezone('Etc/GMT-8'))
print(convert_time(t1, pytz.timezone('Etc/GMT-12')))
print(convert_time(t2, pytz.timezone('Etc/GMT-12')))

01:00:00
01:00:00


似乎，pytz 只对于 datetime 有效，time 无效。

In [8]:
tz_beijing: dt.timezone = dt.timezone(dt.timedelta(hours=8))
tz_trading: dt.timezone = dt.timezone(dt.timedelta(hours=12))

df = df_origin.copy()

print(df.head(20))

                               datetime_nano  last_price  highest   lowest  \
datetime                                                                     
2021-06-30 18:51:42.300  1625050302300000000         NaN      NaN      NaN   
2021-06-30 20:59:00.500  1625057940500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:00.500  1625058000500000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.000  1625058001000000000     18780.0  18780.0  18780.0   
2021-06-30 21:00:01.500  1625058001500000000     18780.0  18785.0  18780.0   
2021-06-30 21:00:02.000  1625058002000000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:02.500  1625058002500000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:03.000  1625058003000000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:03.500  1625058003500000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:04.000  1625058004000000000     18780.0  18795.0  18780.0   
2021-06-30 21:00:04.500  1625058004500000000     18780.0  18795.

In [9]:
df.index = df.index.tz_localize(tz_beijing)
print(df.head(10))

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-06-30 18:51:42.300000+08:00  1625050302300000000         NaN      NaN   
2021-06-30 20:59:00.500000+08:00  1625057940500000000     18780.0  18780.0   
2021-06-30 21:00:00.500000+08:00  1625058000500000000     18780.0  18780.0   
2021-06-30 21:00:01+08:00         1625058001000000000     18780.0  18780.0   
2021-06-30 21:00:01.500000+08:00  1625058001500000000     18780.0  18785.0   

                                   lowest  volume     amount  open_interest  \
datetime                                                                      
2021-06-30 18:51:42.300000+08:00      NaN       0        0.0          10879   
2021-06-30 20:59:00.500000+08:00  18780.0       4   375600.0          10880   
2021-06-30 21:00:00.500000+08:00  18780.0       8   751200.0          10880   
2021-06-30 21:00:01+08:00         18780.0      10   939000

In [10]:
df.index = df.index.tz_convert(tz_trading)
print(df.head(10))

                                        datetime_nano  last_price  highest  \
datetime                                                                     
2021-06-30 22:51:42.300000+12:00  1625050302300000000         NaN      NaN   
2021-07-01 00:59:00.500000+12:00  1625057940500000000     18780.0  18780.0   
2021-07-01 01:00:00.500000+12:00  1625058000500000000     18780.0  18780.0   
2021-07-01 01:00:01+12:00         1625058001000000000     18780.0  18780.0   
2021-07-01 01:00:01.500000+12:00  1625058001500000000     18780.0  18785.0   
2021-07-01 01:00:02+12:00         1625058002000000000     18780.0  18795.0   
2021-07-01 01:00:02.500000+12:00  1625058002500000000     18780.0  18795.0   
2021-07-01 01:00:03+12:00         1625058003000000000     18780.0  18795.0   
2021-07-01 01:00:03.500000+12:00  1625058003500000000     18780.0  18795.0   
2021-07-01 01:00:04+12:00         1625058004000000000     18780.0  18795.0   

                                   lowest  volume     amount  o

In [28]:
index_time = df.index.timetz
print(index_time)
print(type(index_time))
print(index_time[1])
print(index_time[1].tzinfo)

[datetime.time(22, 51, 42, 300000, tzinfo=datetime.timezone(datetime.timedelta(seconds=43200)))
 datetime.time(0, 59, 0, 500000, tzinfo=datetime.timezone(datetime.timedelta(seconds=43200)))
 datetime.time(1, 0, 0, 500000, tzinfo=datetime.timezone(datetime.timedelta(seconds=43200)))
 ...
 datetime.time(4, 59, 59, tzinfo=datetime.timezone(datetime.timedelta(seconds=43200)))
 datetime.time(4, 59, 59, 500000, tzinfo=datetime.timezone(datetime.timedelta(seconds=43200)))
 datetime.time(4, 59, 59, 500001, tzinfo=datetime.timezone(datetime.timedelta(seconds=43200)))]
<class 'numpy.ndarray'>
00:59:00.500000+12:00
UTC+12:00


In [31]:
def to_tz_beijing(t: dt.time) -> dt.time:
    if t.tzinfo == tz_beijing:
        return t
    if t.tzinfo is None:
        return t.replace(tzinfo=tz_beijing)
    temp: dt.datetime = dt.datetime.combine(
        dt.date.today(),
        t
    ).astimezone(tz=tz_trading)
    return temp.astimezone(tz=tz_beijing).timetz()

def to_tz_trading(t: dt.time) -> dt.time:
    if t.tzinfo == tz_trading:
        return t
    if t.tzinfo is None:
        t = t.replace(tzinfo=tz_beijing)
    temp: dt.datetime = dt.datetime.combine(
        dt.date.today(),
        t
    ).astimezone(tz=tz_beijing)
    return temp.astimezone(tz=tz_trading).timetz()

t1: dt.time = dt.time(hour=21, minute=0)
x1: dt.time = to_tz_trading(t1)
print(x1, type(x1), x1.tzinfo)
b1: dt.time = to_tz_beijing(x1)
print(b1, type(b1), b1.tzinfo)

01:00:00+12:00 <class 'datetime.time'> UTC+12:00
21:00:00+08:00 <class 'datetime.time'> UTC+08:00


## 结论

pytz 似乎不大好用，或许没有找到正确方法。

本节的完整程序在 src/timezone.py 中。